### 1.多个文件合并成一个文件

In [2]:
import pandas as pd
from sqlalchemy import create_engine 

In [3]:
'''create_engine()用来初始化数据库连接，括号内为连接信息：
（“数据库类型+数据库驱动名称：//用户名：密码@数据库服务器的名称或IP地址：端口号/
数据库名称?driver=SQL+Server+Native+Client+11.0',encoding='utf-8',echo=False,poolclass=NullPool）'''

engine = create_engine('mssql+pyodbc://:@CLAIRE/baidu_poi?driver=SQL server')

In [4]:
 engine.table_names() 

['baidu_poi_房地产',
 'baidu_poi_公司企业',
 'baidu_poi_购物',
 'baidu_poi_交通设施',
 'baidu_poi_教育培训',
 'baidu_poi_金融',
 'baidu_poi_酒店',
 'baidu_poi_丽人',
 'baidu_poi_旅游景点',
 'baidu_poi_美食',
 'baidu_poi_汽车服务',
 'baidu_poi_生活服务',
 'baidu_poi_文化传媒',
 'baidu_poi_休闲娱乐',
 'baidu_poi_医疗',
 'baidu_poi_运动健身',
 'baidu_poi_政府机构']

In [5]:
def concat_file(table):
    with engine.connect() as conn,conn.begin():
        data = pd.read_sql_table(table,conn)
        return data

In [6]:
df_list = [concat_file(poi) for poi in engine.table_names()]

In [8]:
all_data = pd.concat(df_list)

In [9]:
all_data.head()

,index,name,lat,lng,sort1,sort2
0,0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼
1,1,高新新区,34.173326,108.827928,房地产,写字楼
2,0,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区
3,1,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区
4,2,鑫垚新区,34.174688,108.815573,房地产,住宅区


In [10]:
len(all_data)

178481

### 2.删除重复项

In [13]:
all_data01 = all_data.drop_duplicates(subset = ['name','lat','lng'])

In [14]:
len(all_data01)

140146

In [15]:
all_data01.to_csv(r'../../file/subject/chapter5/all_data.csv')

### 3.坐标转换，BD09转WGS84

长文件，在IDE里写

In [3]:
data = pd.read_csv(r'../../file/subject/chapter5/检查.csv')

In [4]:
data[:5]

,index,name,lat,lng,sort1,sort2
0,0,爱骑行爱生活单车运动协作会,34.170078,108.831835,政府机构,党派团体
1,0,西安市教育考试中心,34.317549,108.944790,政府机构,政治教育机构
2,1,西安市教育考试中心,34.243562,108.958364,政府机构,政治教育机构
3,2,西安教育,34.312222,109.124069,政府机构,政治教育机构
4,3,教育培训基地,34.306449,109.066230,政府机构,政治教育机构


In [5]:
data['lat_wgs84'] = 0.00
data['lng_wgs84'] = 0.00
data

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
0,0,爱骑行爱生活单车运动协作会,34.170078,108.831835,政府机构,党派团体,0.0,0.0
1,0,西安市教育考试中心,34.317549,108.944790,政府机构,政治教育机构,0.0,0.0
2,1,西安市教育考试中心,34.243562,108.958364,政府机构,政治教育机构,0.0,0.0
3,2,西安教育,34.312222,109.124069,政府机构,政治教育机构,0.0,0.0
4,3,教育培训基地,34.306449,109.066230,政府机构,政治教育机构,0.0,0.0
...,...,...,...,...,...,...,...,...
3040,0,西安市灞桥区灞桥街道黄邓社区居委会,34.342635,109.060927,政府机构,居民委员会,0.0,0.0
3041,1,西安市灞桥区灞桥街道康家村村民委员会,34.345189,109.053803,政府机构,居民委员会,0.0,0.0
3042,2,谢一村委会,34.358072,109.061272,政府机构,居民委员会,0.0,0.0
3043,3,中国共产党灞桥区灞桥街道务西村支部委员会,34.351687,109.069085,政府机构,居民委员会,0.0,0.0


In [7]:
data02 = data.copy()

In [74]:
#定义一个函数，能够修改pandas对象的值

In [20]:
len(data02)%100

45

In [58]:
for i in range(len(data02)-(len(data02)%100),len(data02)):
    data02.loc[i:i,('lat_wgs84','lng_wgs84')] = [i-1,i+1]

In [71]:
data02[3000:3003]

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
3000,2,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,社会团体,2999,3001.0
3001,3,陕西省国际标准舞学会,34.353181,108.974071,政府机构,社会团体,3000,3002.0
3002,4,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,社会团体,3001,3003.0


In [75]:
import time
#前100整数个对象
start_time = time.time()
for i in range(len(data02)//100+1):
    def coor_trans():
        for j in  range(i*100):
            data02.loc[j:j,('lat_wgs84','lng_wgs84')] = [j-2,j+2]
    coor_trans()
#最后一个非100整数对象
for i in range(len(data02)-(len(data02)%100),len(data02)):
    data02.loc[i:i,('lat_wgs84','lng_wgs84')] = [i-2,i+2]
end_time = time.time()
print('time use:{}'.format(end_time-start_time))

time use:45.426859617233276


In [76]:
data02.loc[2995:3003]

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
2995,5,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,党派团体,2993,2997.0
2996,6,中共五龙汤社区支部委员会,34.356650,108.975046,政府机构,党派团体,2994,2998.0
2997,7,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,党派团体,2995,2999.0
2998,8,中国共产党西安未央康乐医院支部委员会,34.352732,108.971605,政府机构,党派团体,2996,3000.0
2999,1,西安市示范基层老年协会,34.356742,108.975105,政府机构,社会团体,2997,3001.0
3000,2,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,社会团体,2998,3002.0
3001,3,陕西省国际标准舞学会,34.353181,108.974071,政府机构,社会团体,2999,3003.0
3002,4,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,社会团体,3000,3004.0
3003,5,航空行业第十三职业技能鉴定站操作技能培训考试基地,34.357061,108.972291,政府机构,社会团体,3001,3005.0
